In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

In [ ]:
test = pd.read_csv('../input/test.csv')
item_categories = pd.read_csv('../input/item_categories.csv')
items = pd.read_csv('../input/items.csv')
shops = pd.read_csv('../input/shops.csv')
train = pd.read_csv('../input/sales_train.csv', parse_dates=['date'])

In [ ]:
train.head()

In [ ]:
items.head()

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns 
import altair as alt 

In [ ]:
train = train[train['item_cnt_day'] < 2000]
train = train[train['item_price'] < 300000]

price_correction = train[(train['shop_id'] == 32) & (train['item_id'] == 2973) & (train['date_block_num'] == 4) & (train['item_price'] > 0)].item_price.median()
train.loc[train['item_price'] < 0, 'item_price'] = price_correction



In [ ]:
train.loc[train['shop_id'] == 0, 'shop_id'] = 57
test.loc[test['shop_id'] == 0, 'shop_id'] = 57

train.loc[train['shop_id'] == 1, 'shop_id'] = 58
test.loc[test['shop_id'] == 1, 'shop_id'] = 58

train.loc[train['shop_id'] == 10, 'shop_id'] = 11
test.loc[test['shop_id'] == 10, 'shop_id'] = 11



In [ ]:
cities = shops['shop_name'].str.split(' ').map(lambda row: row[0])
cities.unique()

In [ ]:
shops['city'] = shops['shop_name'].str.split(' ').map(lambda row: row[0])
shops.loc[shops.city == '!Якутск', 'city'] = 'Якутск'

from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit_transform(shops['city'])

shops['city_label'] = le.fit_transform(shops['city'])
shops.drop(['shop_name', 'city'], axis = 1, inplace = True)
shops.head()

In [ ]:
le = preprocessing.LabelEncoder()

main_categories = item_categories['item_category_name'].str.split('-')
item_categories['main_category_id'] = main_categories.map(lambda row: row[0].strip())
item_categories['main_category_id'] = le.fit_transform(item_categories['main_category_id'])

# Some items don't have sub-categories. For those, we will use the main category as a sub-category
item_categories['sub_category_id'] = main_categories.map(lambda row: row[1].strip() if len(row) > 1 else row[0].strip())
item_categories['sub_category_id'] = le.fit_transform(item_categories['sub_category_id'])


In [ ]:
item_categories.head()


In [ ]:
train.head()

In [ ]:
x = train.groupby(['shop_id', 'item_id', 'date_block_num'])['item_cnt_day'].sum().rename('item_cnt_month').reset_index()
test.insert(loc=3, column='date_block_num', value=34)

In [ ]:
x.head()

In [ ]:
test.head()

In [ ]:
test['item_cnt_month'] = 0

In [ ]:
new_train = x.append(test.drop('ID', axis = 1))

In [ ]:
new_train = pd.merge(new_train, shops, on=['shop_id'], how='left')
new_train.head()

In [ ]:
new_train = pd.merge(new_train, items.drop('item_name', axis = 1), on=['item_id'], how='left')
new_train.head()

In [30]:
new_train = pd.merge(new_train, item_categories.drop('item_category_name', axis = 1), on=['item_category_id'], how='left')
new_train.head()

,shop_id,item_id,date_block_num,item_cnt_month,city_label,item_category_id,main_category_id_x,sub_category_id_x,main_category_id_y,sub_category_id_y,main_category_id_x,sub_category_id_x,main_category_id_y,sub_category_id_y,main_category_id_x,sub_category_id_x,main_category_id_y,sub_category_id_y,main_category_id,sub_category_id
0,2,27,0,1.0,0,19,5,10,5,10,5,10,5,10,5,10,5,10,5,10
1,2,27,17,1.0,0,19,5,10,5,10,5,10,5,10,5,10,5,10,5,10
2,2,30,2,1.0,0,40,11,4,11,4,11,4,11,4,11,4,11,4,11,4
3,2,30,5,1.0,0,40,11,4,11,4,11,4,11,4,11,4,11,4,11,4
4,2,30,15,1.0,0,40,11,4,11,4,11,4,11,4,11,4,11,4,11,4


In [ ]:

import xgboost as xgb
def xgtrain():
    regressor = xgb.XGBRegressor(n_estimators = 5000,
                                 learning_rate = 0.01,
                                 max_depth = 10,
                                 subsample = 0.5,
                                 colsample_bytree = 0.5)
    
    regressor_ = regressor.fit(new_train[new_train.date_block_num < 33].drop(['item_cnt_month'], axis=1).values, 
                               new_train[new_train.date_block_num < 33]['item_cnt_month'].values, 
                               eval_metric = 'rmse', 
                               eval_set = [(new_train[new_train.date_block_num < 33].drop(['item_cnt_month'], axis=1).values, 
                                            new_train[new_train.date_block_num < 33]['item_cnt_month'].values), 
                                           (new_train[new_train.date_block_num == 33].drop(['item_cnt_month'], axis=1).values, 
                                            new_train[new_train.date_block_num == 33]['item_cnt_month'].values)
                                          ], 
                               verbose=True,
                               early_stopping_rounds = 50,
                              )
    return regressor_


In [ ]:
%%time
regressor_ = xgtrain()